In [9]:
import json
import deep_translator
import pandas as pd
import streamlit as st
from num2words import num2words
from tqdm.notebook import tqdm

print("pandas version: ", pd.__version__)
print("streamlit version: ", st.__version__)
print("json version: ", json.__version__)
print("deep_translator version: ", deep_translator.__version__)

pandas version:  1.5.3
streamlit version:  1.20.0
json version:  2.0.9
deep_translator version:  1.9.1


In [10]:
def dutch_to_english(text, batch=False):
    '''
    Translate dutch to english
    
    :param text: text to translate
    :param batch: if batch is True, translate a list of texts
    '''
    if len(text) > 1:
        try:
            if batch:
                keywords = []
                for words in text:
                    try:
                        keywords.append(GoogleTranslator(
                            source='auto', target='en').translate(words))
                    except:
                        num_word = num2words(float(words))
                        keywords.append(GoogleTranslator(
                            source='auto', target='en').translate(num_word))
                return keywords
            return GoogleTranslator(source='nl', target='en').translate(text)
        except:
            return text
    else:
        return np.nan

In [11]:
def to_datetime(df, cols, format):
    '''
    Function to convert a column to datetime 

    :param df: pandas dataframe
    :param cols: column to convert
    :param format: format of the date time
    '''
    def to_datetime(col):
        return pd.to_datetime(col, format = format)
    df[cols] = df[cols].apply(to_datetime)
    return df

In [12]:
# create function to get the content id for each rating

def get_id(dt, df_c):
    '''
    Function to get the content id for each rating

    :param dt: datetime of the rating
    :param df_c: content dataframe
    '''
    mask = (dt > df_c['date_time_start']) & (dt <= df_c['date_time_end'])
    if mask.any():
        return df_c[mask]['id'].values[0]

# Data Pre-processing

In [ ]:
# preprocess the content data

df_content_processed = (pd.read_pickle('banijay_op1data_content_raw.pkl')
                        .rename(columns=str.lower)
                        .assign(
                            #title_en = lambda df: df['title'].apply(dutch_to_english),
                            #summary_en = lambda df: df['summary'].apply(dutch_to_english),
                            #keywords_en = lambda df: df['keywords'].apply(dutch_to_english, batch=True),
                            date_time_start = lambda df: (df['date'] + " " + df['start'].str[:-3]),
                            date_time_end = lambda df: (df['date'] + " " + df['end'].str[:-3]),)
                        .pipe(to_datetime, ['date_time_start', 'date_time_end'], '%d-%m-%Y %H:%M:%S')
                        .drop(['date', 'start', 'end'], axis=1)
                        .drop_duplicates(subset=['id'])
                        .dropna(subset=['id'])
                        )

print("Content data: ", df_content_processed.shape)

In [ ]:
# preprocess the ratings data

df_ratings_processed = (pd.read_csv('banijay_op1data_ratings_raw.csv', sep=";", decimal=",")
                        .rename(columns=str.lower)
                        .assign(
                            date_time = lambda df: (df['datum'] + " " + df['time']))
                        .pipe(to_datetime, ['date_time'], '%Y-%m-%d %H:%M:%S')
                        .drop(['datum', 'time'], axis=1)
                        .dropna(subset=['date_time'])
                        )

print("Ratings data: ", df_ratings_processed.shape)

In [ ]:
# create a lookup table for the content id (id) and the datetime (date_time
# this is useful because the index of the ratings dataframe is not
# the same as the index of the content dataframe

df_ratings_lookup =  (pd.DataFrame(df_ratings_processed['date_time'].unique(), columns=['date_time'])
                        .assign(
                            id = lambda df: df['date_time'].apply(get_id, df_c=df_content_processed))
                        )   

print("Lookup table: ", df_ratings_lookup.shape)

In [ ]:
# Merge the ratings and content dataframes using the lookup table

df_merged = (df_ratings_processed
                .merge(df_ratings_lookup, on='date_time', how='left', validate='many_to_one')
                .merge(df_content_processed, on='id', how='left', validate='many_to_one')
                .drop(['program','station','kdh%','zadl%'], axis=1)
                .dropna(subset=['id'])
             )

print("Merged Data: ", df_merged.shape)

# Data Exploration

# get mean kdh000 group by target group for ratings type totaal

(df_merged
    .loc[df_merged['ratings type'] == 'totaal', :]
    .groupby(['target group'])['kdh000']
    .mean()
    .sort_values(ascending=False)
)

In [ ]:
# Barplot of the mean kdh000 per target group

(df_merged
    .loc[df_merged['ratings type'] == 'totaal', :]
    .groupby(['target group'])['kdh000']
    .mean()
    .sort_values(ascending=False)
    .plot(kind='bar', 
          figsize=(10,5), 
          title='Mean kdh000 per target group based on ratings type totaal',
          grid=True,
          color='orange',
          legend=True)
)

In [ ]:
 Monthly trend analysis

(df_merged
    .assign(
        month = lambda x: x['date_time'].dt.month)
    .loc[df_merged['ratings type'] == 'totaal', :]
    .groupby(['month'])['kdh000']
    .mean()
    .plot(kind='line',
            figsize=(10,5),
            title='Monthly trend analysis',
            grid=True,
            color='orange',
            legend=True)
)